In [69]:
with open('curp_duplicados (1).csv') as f:
    data = f.readlines()[1:]
    data = [row[:-1].split(',')[-2:] for row in data]
data

[['GACR930304HNLRRF06', '72682'],
 ['GAVJ940723HNLRLS09', '77215'],
 ['GAVJ940723HNLRLS09', '77660'],
 ['RASO690402MNLMPF02', '73030'],
 ['RASO690402MNLMPF02', '35496'],
 ['RASO690402MNLMPF02', '73936'],
 ['ROCC020606MNLDRNA4', '69673'],
 ['ROER571116HTSDSY11', '54261'],
 ['ROER571116HTSDSY11', '66075'],
 ['ROGL670310HNLDVC00', '73174'],
 ['ROGL670310HNLDVC00', '73117'],
 ['ROHD140527MNLDRNA1', '72087'],
 ['ROHD140527MNLDRNA1', '53942'],
 ['RORJ751121HTSSSN03', '67609'],
 ['RORJ751121HTSSSN03', '65375'],
 ['ROTY030926HNLDVRA4', '68834'],
 ['ROTY030926HNLDVRA4', '46249'],
 ['ROTY111020HNLDVSA4', '46249'],
 ['ROTY111020HNLDVSA4', '68834'],
 ['ROVI860217MNLDLR08', '56618'],
 ['ROVI860217MNLDLR08', '73259'],
 ['RUAL580120HTSZRB04', '71498'],
 ['RUAL580120HTSZRB04', '71221'],
 ['SACJ150708HNLCSSA7', '71152'],
 ['SACJ150708HNLCSSA7', '74115'],
 ['SACV160819MNLCSCA0', '74115'],
 ['SACV160819MNLCSCA0', '71152'],
 ['SAGP200808MNLLZLA9', '68204'],
 ['SAGP200808MNLLZLA9', '68202'],
 ['SAGV000101M

In [70]:
from sqlalchemy import create_engine
import pandas as pd



def connect_ps():

    SERVER = '10.153.170.46'
    DATABASE = 'IntegraGrupos'
    DRIVER = 'SQL Server'
    USERNAME = 'sa'
    PASSWORD = 'desarrollo'
    DATABASE_CONNECTION = f'mssql://{USERNAME}:{PASSWORD}@{SERVER}/{DATABASE}?driver={DRIVER}'

    engine = create_engine(DATABASE_CONNECTION)
    connection = engine.connect()

    return connection

pd.set_option('display.max_columns', None)

connection = connect_ps()

# Checamos viviendas donde vive la persona

In [111]:
query = '''
SELECT db.IDBeneficiario, db.IDVivienda, db.CURP, bc.IDEstatusBeneficiario FROM DatosBeneficiario AS db
LEFT JOIN BeneficiarioControl AS bc ON db.IDBeneficiario = bc.IDBeneficiario 
WHERE CURP='RAHL731106MSPMNRT0'
'''

initial_df = pd.read_sql_query(query, connection)
initial_df

,IDBeneficiario,IDVivienda,CURP,IDEstatusBeneficiario
0,226126,71185,RAHL731106MSPMNRT0,NaN
1,226375,71277,RAHL731106MSPMNRT0,4.0


# Revisamos si esas viviendas tienen algun beneficiario

Primero obtenemos el universo de estas viviendas

In [112]:
id_viviendas = initial_df['IDVivienda'].to_list()
id_viviendas = [str(id) for id in id_viviendas]
id_viviendas_str = "'"+"','".join(id_viviendas)+"'"

query = f'''
SELECT db.CURP, db.IDVivienda, bc.IDEstatusBeneficiario
FROM DatosBeneficiario AS db
LEFT JOIN BeneficiarioControl AS bc ON db.IDBeneficiario = bc.IDBeneficiario 
WHERE IDVivienda in ({id_viviendas_str})
'''

viviendas_df = pd.read_sql_query(query, connection)
viviendas_df

,CURP,IDVivienda,IDEstatusBeneficiario
0,BARS141212MNLTMCA4,71185,NaN
1,BARA160330MNLTMNA9,71185,NaN
2,RAHL731106MSPMNRT0,71185,NaN
3,BARS141212MNLTMCA4,71277,NaN
4,BARA160330MNLTMNA9,71277,NaN
5,RAHL731106MSPMNRT0,71277,4.0


Luego revisamos de estas viviendas, cuáles tienen algun beneficiario

In [113]:
viviendas_con_beneficiaro = viviendas_df.loc[viviendas_df['IDEstatusBeneficiario'] == 4.0]
viviendas_con_beneficiaro

,CURP,IDVivienda,IDEstatusBeneficiario
5,RAHL731106MSPMNRT0,71277,4.0


Luego, obtenemos como vivienda válida aquella que tiene algún beneficiario.

Si hay menos de una, la vivienda válida es la más reciente

In [115]:
if viviendas_con_beneficiaro.shape[0] == 1:
    vivienda_valida = int(viviendas_con_beneficiaro['IDVivienda'].iloc[0])
elif viviendas_con_beneficiaro.shape[0] == 0:
    vivienda_valida = viviendas_df['IDVivienda'].max()
vivienda_a_eliminar = viviendas_df.loc[viviendas_df['IDVivienda'] != vivienda_valida]['IDVivienda'].unique()
vivienda_a_eliminar

array([71185], dtype=int64)

# Ahora ensamblamos todo

In [75]:
viviendas_a_eliminar_list = []

curp_list = [row[0] for row in data]

curp_str = "'"+"','".join(curp_list)+"'"

query = f'''
SELECT db.IDBeneficiario, db.IDVivienda, db.CURP, bc.IDEstatusBeneficiario FROM DatosBeneficiario AS db
LEFT JOIN BeneficiarioControl AS bc ON db.IDBeneficiario = bc.IDBeneficiario 
WHERE CURP IN ({curp_str})
'''

initial_df = pd.read_sql_query(query, connection)

id_viviendas = initial_df['IDVivienda'].to_list()
id_viviendas = [str(id) for id in id_viviendas]
id_viviendas_str = "'"+"','".join(id_viviendas)+"'"

query = f'''
SELECT db.IDVivienda, bc.IDEstatusBeneficiario
FROM DatosBeneficiario AS db
LEFT JOIN BeneficiarioControl AS bc ON db.IDBeneficiario = bc.IDBeneficiario 
WHERE IDVivienda in ({id_viviendas_str})
'''

viviendas_df = pd.read_sql_query(query, connection)

viviendas_con_beneficiaro = viviendas_df.loc[viviendas_df['IDEstatusBeneficiario'] == 4.0]

if viviendas_con_beneficiaro.shape[0] == 1:
    vivienda_valida = int(viviendas_con_beneficiaro['IDVivienda'])
elif viviendas_con_beneficiaro.shape[0] == 0:
    vivienda_valida = viviendas_df['IDVivienda'].max()
vivienda_a_eliminar = viviendas_df.loc[viviendas_df['IDVivienda'] != vivienda_valida]['IDVivienda'].unique()
vivienda_a_eliminar

array([40706, 67609, 54426, 70680, 72721, 72796, 68847, 69134, 70041,
       79530, 78624, 35497, 68165, 65684, 28320, 68135, 67290, 67293,
       79531, 71994, 72788, 67268, 65960, 39637, 46249, 73936, 72682,
       71119, 71472, 74115, 79194, 60910, 68219, 65161, 66076, 66220,
       72790, 45505, 76979, 56618, 68673, 74227, 68661, 69673, 73007,
       29038, 58377, 79529, 78112, 73117, 65683, 79907, 67291, 63160,
       70928, 67025,  3038, 66075, 72014,  4301, 65159, 65375, 78626,
       66229, 73855, 66211, 70018, 79903, 35496, 68511, 71918, 53942,
       54261, 71884, 67026, 68834, 70181, 71152, 77215, 70381, 78625,
       72087, 71063, 62796, 68202, 22280, 71498, 70547, 72019, 66692,
       79355, 65891, 72009, 72325, 71883, 73259, 72787, 68204, 71221,
       73030, 73174], dtype=int64)

In [76]:
len(vivienda_a_eliminar)

101

In [118]:
import pandas as pd

# Obtén todos los datos necesarios en una sola consulta SQL que también agrupa por CURP y IDVivienda
query = '''
SELECT db.CURP, db.IDVivienda, bc.IDEstatusBeneficiario
FROM DatosBeneficiario AS db
LEFT JOIN BeneficiarioControl AS bc ON db.IDBeneficiario = bc.IDBeneficiario
WHERE CURP IN (SELECT CURP FROM DatosBeneficiario GROUP BY CURP)
AND CURP != ''
ORDER BY db.CURP, db.IDVivienda, bc.IDEstatusBeneficiario DESC
'''
df = pd.read_sql_query(query, connection)

# Inicializa una lista para almacenar las viviendas a eliminar
viviendas_a_eliminar = []

# Procesa el DataFrame por CURP
for curp, group in df.groupby('CURP'):
    # Revisa si hay alguna vivienda con IDEstatusBeneficiario == 4.0
    viviendas_con_beneficiario = group[group['IDEstatusBeneficiario'] == 4.0]
    
    if not viviendas_con_beneficiario.empty:
        # Si existe alguna vivienda con estado 4.0, selecciona la primera (debido al ordenamiento)
        vivienda_valida = viviendas_con_beneficiario['IDVivienda'].iloc[0]
    else:
        # Si no hay ninguna vivienda con estado 4.0, toma el IDVivienda máximo de ese CURP
        vivienda_valida = group['IDVivienda'].max()
    
    # Añade todas las viviendas menos la válida a la lista de eliminación
    viviendas_a_eliminar.extend(group[group['IDVivienda'] != vivienda_valida]['IDVivienda'].unique())

# Elimina duplicados en la lista de viviendas a eliminar
viviendas_a_eliminar = list(set(viviendas_a_eliminar))

print(viviendas_a_eliminar)


[32774, 73734, 65547, 73740, 8205, 32780, 15, 16, 17, 12, 49170, 21, 73749, 81941, 16412, 73759, 32, 73761, 81955, 32803, 73767, 42, 24620, 45, 49197, 48, 73779, 65588, 32821, 57404, 65597, 62, 63, 73792, 66, 67, 73796, 57415, 73800, 73799, 75, 41037, 16464, 16465, 49234, 73810, 65633, 102, 65646, 73839, 73841, 82034, 82035, 82036, 65653, 73855, 82048, 49281, 130, 16514, 24707, 73861, 73865, 82059, 65683, 65684, 57495, 65688, 57499, 82080, 65697, 41126, 57512, 41130, 57516, 41140, 65716, 65720, 82105, 16576, 41153, 82115, 73924, 73925, 65733, 65732, 82120, 65736, 65741, 32981, 65752, 73945, 73946, 32994, 24805, 82153, 82159, 82163, 82165, 246, 49405, 82175, 82176, 82177, 82178, 82179, 259, 73988, 82183, 49416, 82185, 65802, 16648, 268, 8460, 270, 49424, 272, 49428, 277, 65822, 49440, 24865, 41250, 49445, 24870, 74023, 49450, 65836, 307, 49460, 308, 82230, 49462, 49465, 65853, 49471, 49473, 49474, 65858, 24898, 65859, 65863, 49480, 65868, 49489, 49490, 41298, 65880, 82265, 74074, 49499,

In [121]:
len(viviendas_a_eliminar)

3135

In [105]:
counter = 0
for curp in df.groupby('CURP'):
    if curp[1].shape[0] > 1:
        counter +=1
        print(curp)
print(counter)

('AAAA890207HNLMLR05',                   CURP  IDVivienda  IDEstatusBeneficiario
65  AAAA890207HNLMLR05       67196                    NaN
66  AAAA890207HNLMLR05       67197                    NaN)
('AAAD220105MNLMGLA5',                    CURP  IDVivienda  IDEstatusBeneficiario
149  AAAD220105MNLMGLA5       65047                    NaN
150  AAAD220105MNLMGLA5       65461                    NaN)
('AAAE680814MNLMLL08',                    CURP  IDVivienda  IDEstatusBeneficiario
181  AAAE680814MNLMLL08       15782                    4.0
182  AAAE680814MNLMLL08       15782                    4.0)
('AAAE950523MNLMRS05',                    CURP  IDVivienda  IDEstatusBeneficiario
189  AAAE950523MNLMRS05       29221                    5.0
190  AAAE950523MNLMRS05       65163                    4.0)
('AAAF130615HNLLLBA6',                    CURP  IDVivienda  IDEstatusBeneficiario
196  AAAF130615HNLLLBA6       36795                    NaN
197  AAAF130615HNLLLBA6       71237                    NaN

In [103]:
curp[1].shape

(1, 3)

In [91]:
print(vivienda_a_eliminar.shape)

(101,)


# Validación

In [79]:
id_vivienda_list = [int(row[1]) for row in data]
id_vivienda_list

[72682,
 77215,
 77660,
 73030,
 35496,
 73936,
 69673,
 54261,
 66075,
 73174,
 73117,
 72087,
 53942,
 67609,
 65375,
 68834,
 46249,
 46249,
 68834,
 56618,
 73259,
 71498,
 71221,
 71152,
 74115,
 74115,
 71152,
 68204,
 68202,
 67026,
 67025,
 67291,
 67290,
 22280,
 70381,
 22280,
 70381,
 72014,
 72009,
 72788,
 72787,
 79907,
 79903,
 40706,
 73855,
 68511,
 60910,
 65960,
 62796,
 46249,
 68834,
 66229,
 4301,
 39637,
 66211,
 71918,
 72325,
 72325,
 71918,
 68165,
 68135,
 69134,
 71883,
 70547,
 68673,
 54261,
 66075,
 79529,
 79530,
 79529,
 79530,
 79529,
 79530,
 65161,
 65159,
 65161,
 65159,
 65891,
 65683,
 68511,
 60910,
 79194,
 79355,
 78624,
 78626,
 70928,
 63160,
 56618,
 73259,
 70018,
 70041]

In [80]:
len(id_vivienda_list)

91

In [81]:
set(id_vivienda_list)-set(vivienda_a_eliminar)

{77660}